# VAE について書く
- https://www.youtube.com/watch?v=g5RECWW-7Wg
- 実行できない。。。書写しをミスした？

In [1]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(1)

In [2]:
def select_device():
    # “”"GPU もしくは CPU の選択“”"
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print('cuda is selected as device!')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
        print('mps is selected as device!')
    else:
        device = torch.device('cpu')
        print('cpu....f')
    return device
device = select_device()

mps is selected as device!


In [3]:
transform = transforms.Compose([transforms.ToTensor(), lambda x: x.view(-1)])

root = './data_tutorial_VAE'
mnist_dataset = datasets.MNIST(root=root, download=True, train=True, transform=transform)
dataloader = DataLoader(mnist_dataset, batch_size=100, shuffle=True)

In [4]:
class VAE(nn.Module):
    def __init__(self, device = 'mps'):
        super().__init__()
        self.device = device
        self.encoder = Encoder(device=device)
        self.decoder = Decoder(device=device)
    
    def forward(self, x):
        mean, var = self.encoder(x)
        z = self.reparameterize(mean, var)
        y = self.decoder(z)
        return y, z

    def reparameterize(self, mean, var):
        z = mean + torch.sqrt(var) * torch.randn(mean.size()).to(self.device)
        return z
    
    def criterion(self, x):
        mean, var = self.encoder(x)
        z = self.reparameterize(mean, var)
        y = self.decoder(z)
        L1 = - torch.mean(torch.sum(x * torch.log(y) + (1 - x) * torch.log(1 - y), dim=1))
        L2 = - 1/2 * torch.mean(torch.sum(1 + torch.log(var) - mean**2 - var, dim=1))
        L = L1 + L2
        return L



In [5]:
class Encoder(nn.Module):
    def __init__(self, device = 'mps'):
        super().__init__()
        self.device = device
        self.l1 = nn.Linear(784, 256)
        self.l2 = nn.Linear(256, 128)
        self.l_mean = nn.Linear(128, 2)
        self.l_var = nn.Linear(128, 2)

    def forward(self, x):
        h = self.l1(x)
        h = torch.relu(h)
        h = self.l2(h)
        h = torch.relu(h)
        mean = self.l_mean(h)
        var = self.l_var(h)
        var = F.softplus(var)

        return mean, var

In [6]:
class Decoder(nn.Module):
    def __init__(self, device='mps'):
        super().__init__()
        self.device = device
        self.l1 = nn.Linear(2, 128)
        self.l2 = nn.Linear(128, 256)
        self.out = nn.Linear(256, 784)

    def forward(self, x):
        h = self.l1(x)
        h = torch.relu(h)
        h = self.l2(h)
        h = torch.relu(h)
        h = self.out(h)
        y = torch.sigmoid(h)

        return y
        

In [7]:

model = VAE(device=device).to(device)
criterion = model.criterion
optimizer = optim.Adam(model.parameters())

In [8]:
n_epoch = 8

for epoch in range(n_epoch):
    loss_mean = 0
    for (x, t) in dataloader:
        x = x.to(device)
        model.train()
        print('loss criterion')
        loss = criterion(x)
        print('zero')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_mean += loss.item()
    loss_mean /= len(dataloader)
        

x, t
train
loss criterion


: 

: 

In [ ]:
model.eval()
z = torch.randn(10, 2, device = device)
images = model.decoder(z)
images = images.view(-1, 28, 28)
images = images.squeeze().detach().cpu().numpy()

# データの可視化
for i, image in enumerate(images):
    plt.subplot(2, 5, i+1)
    plt.imshow(image, cmap='binary_r')
    plt.axis('off')
plt.tight_layout()
plt.show()

NameError: name 'model' is not defined

In [ ]:
# データ可視化の前準備
img_size = 28
n_image = 10
image_size_spaced = img_size +2
matrix_image = np.zeros((image_size_spaced*n_image, image_size_spaced*n_image))

z_1 = torch.linspace(-3, 3, n_image)
z_2 = torch.linspace(-3, 3, n_image)

for i, z1 in enumerate(z_1):
    for j, z2 in enumerate(z_2):
        x = torch.tensor([float(z1), float(z2)], device=device)
        images = model.decoder(x)
        images = images.view(-1, 28, 28)
        images = images.squeeze().detach().cpu().numpy()
        top = i * image_size_spaced
        left = j * image_size_spaced
        matrix_image[top : top + img_size, left: left+img_size] = images

plt.figure(figsize=(8, 8))
plt.how(matrix_image.tolist(), cmap="Greys_r")
plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)
plt.show()